In [38]:
!pip install pandas
!pip install requests
!pip install beautifulsoup4
!pip install tqdm
!pip install yahoo_fin


In [39]:
import datetime
import pandas as pd # pip install pandas
import requests # pip install requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import yfinance as yf
from tqdm import tqdm
from yahoo_fin import options

In [40]:
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
}
url = 'https://www.marketwatch.com/tools/earningscalendar'
r = requests.get(url, headers = headers)
# Request with url and header
soup = BeautifulSoup(r.content, 'html.parser')
tabpane = soup.find('div', 'tabpane')
earning_tables = tabpane.find_all('div', 'daypage')
current_datetime = datetime.date.today() + datetime.timedelta(days = 1) 
tomorrow = current_datetime.strftime('%b_%d')
anticipated_earnings = pd.DataFrame()
print('Preparing to Scrap Data...')
for earning_table in earning_tables:
    earning_date = earning_table['id'].replace('page', '')
    earning_date = earning_date[:3] + '_' + earning_date[3:]
    if tomorrow == earning_date:
        table = earning_table.find("table")
        pretty = table.prettify()
        pd_table = pd.read_html(pretty)
        pd_table_1 = pd_table[0]
        anticipated_earnings = pd_table_1
print('Finished Scraping Data...')
anticipated_earnings.columns = ["Company Name", "Symbol", "Fiscal Quarter", "EPS Forecast", "EPS Actual", "Suprise"]
pd.set_option("display.max_rows", None, "display.max_columns", None)
anticipated_earnings = anticipated_earnings.drop(columns=["Suprise", "EPS Actual", "Fiscal Quarter","EPS Forecast"])
cash = 2932
eligible_price = float(cash / 100)
eligible_tickers = anticipated_earnings
print('Finished Processing Data...')

Preparing to Scrap Data...
Finished Scraping Data...
Finished Processing Data...


In [41]:
eligible_tickers

,Company Name,Symbol
0,Rollins Inc.,ROL
1,Advantest Corp. ADR,ATEYY
2,DSV A/S ADR,DSDVY
3,American Water Works Co.,AWK
4,Daiwa Securities Group Inc. ADR,DSEEY
5,NeoGenomics Inc.,NEO
6,BioMarin Pharmaceutical Inc.,BMRN
7,Astellas Pharma Inc. ADR,ALPMY
8,Akbank T.A.S. ADR,AKBTY
9,Deutsche Bank AG,DB


In [42]:
today_formatted = datetime.date.today().strftime("%Y-%m-%d")
print('Processing Stock Tickers...')
for index, row in eligible_tickers.iterrows():
    try:
        current_ticker = float(yf.download(row['Symbol'],today_formatted, progress=False)['Adj Close'])
        if (current_ticker > eligible_price):
            eligible_tickers.drop(index, inplace=True)
            print('Dropping ' + row['Symbol'] + ' ' + str(current_ticker))
#         else:
#             eligible_tickers.iat[index,3] = current_ticker
#             print(row['Symbol'] + 'current price' + current_ticker)
    except:
        eligible_tickers.drop(index, inplace=True)
print('Finished Processing Stock Tickers...')

Processing Stock Tickers...
Dropping ROL 32.720001220703125
Dropping ATEYY 65.55999755859375
Dropping DSDVY 80.55000305175781
Dropping AWK 158.25999450683594
Dropping BMRN 79.29000091552734
Dropping HKXCY 40.290000915527344
Dropping BA 167.0399932861328
Dropping HINOY 51.5099983215332
Dropping HESM 29.770000457763672
Dropping HES 101.4800033569336

1 Failed download:
- AKBDY: No data found, symbol may be delisted
Dropping DASTY 41.31999969482422
Dropping IBDRY 44.93000030517578
Dropping SHOO 42.310001373291016
Dropping MEOH 48.18000030517578
Dropping NVZMY 67.05999755859375
Dropping ICLR 210.6699981689453
Dropping FORM 35.61000061035156
Dropping R 67.95999908447266
Dropping TPB 31.0
Dropping DDAIF 66.94000244140625
Dropping CCEP 50.72999954223633
Dropping GSK 44.38999938964844
Dropping STM 36.779998779296875
Dropping HUM 430.5

1 Failed download:
- CYJBY: No data found for this date range, symbol may be delisted
Dropping EVR 107.29000091552734
Dropping MAS 52.72999954223633
Dropping CM

In [43]:
print(eligible_tickers)

                                          Company Name Symbol
4                      Daiwa Securities Group Inc. ADR  DSEEY
5                                     NeoGenomics Inc.    NEO
7                             Astellas Pharma Inc. ADR  ALPMY
8                                    Akbank T.A.S. ADR  AKBTY
9                                     Deutsche Bank AG     DB
10                   United Microelectronics Corp. ADR    UMC
13                      China Coal Energy Co. Ltd. ADR  CCOZY
14                       Renesas Electronics Corp. ADR  RNECY
23                   China Life Insurance Co. Ltd. ADR    LFC
26                        Lloyds Banking Group PLC ADR    LYG
27         SINOPEC Shanghai Petrochemical Co. Ltd. ADR    SHI
28                                        Alkermes PLC   ALKS
29                                 Cenovus Energy Inc.    CVE
30                                  MDxHealth S.A. ADR   MDXH
31                                Mastech Digital Inc.    MHH
34      

In [50]:
# raw_data = []
# for row in eligible_tickers.iterrows():
#     current_ticker = float(yf.download(row['Symbol'],today_formatted, progress=False)['Adj Close'])  
#     dates = options.get_expiration_dates(row['Symbol'])
#     chain = options.get_options_chain(row['Symbol'],dates[0])["puts"]
dates = options.get_expiration_dates("PINS")
chain = options.get_options_chain("PINS",dates[0])["puts"]
print(chain)

          Contract Name         Last Trade Date  Strike  Last Price  Bid  Ask  \
0   PINS220429P00014000   2022-04-26 3:03PM EDT    14.0        0.11  0.0  0.0   
1   PINS220429P00014500   2022-04-26 2:10PM EDT    14.5        0.15  0.0  0.0   
2   PINS220429P00015000   2022-04-26 3:52PM EDT    15.0        0.20  0.0  0.0   
3   PINS220429P00015500   2022-04-26 3:36PM EDT    15.5        0.25  0.0  0.0   
4   PINS220429P00016000   2022-04-26 3:56PM EDT    16.0        0.35  0.0  0.0   
5   PINS220429P00016500   2022-04-26 3:55PM EDT    16.5        0.45  0.0  0.0   
6   PINS220429P00017000   2022-04-26 3:58PM EDT    17.0        0.58  0.0  0.0   
7   PINS220429P00017500   2022-04-26 3:58PM EDT    17.5        0.73  0.0  0.0   
8   PINS220429P00018000   2022-04-26 3:59PM EDT    18.0        0.91  0.0  0.0   
9   PINS220429P00018500   2022-04-26 3:59PM EDT    18.5        1.10  0.0  0.0   
10  PINS220429P00019000   2022-04-26 3:59PM EDT    19.0        1.31  0.0  0.0   
11  PINS220429P00019500   20

0     0.005
1     0.010
2     0.005
3     0.010
4     0.010
5     0.005
6     0.015
7     0.025
8     0.020
9     0.025
10    0.025
11    0.035
12    0.045
13    0.045
14    0.045
15    0.045
16    0.075
17    0.075
18    0.075
19    0.100
20    0.125
21    0.075
22    0.150
23    0.150
24    0.100
25    0.100
26    0.100
27    0.200
28    0.100
29    0.225
30    0.375
31    0.100
32    0.075
33    0.150
34    0.125
35    0.175
36    0.100
37    0.100
38    0.125
39    0.175
40    0.125
Name: mid_price, dtype: float64
